In [96]:
# required modules
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('default')

In [97]:
df_rest = pd.read_csv('../data/data_combined_v1.00.csv')
df_rest.head()

,place_id,tripadvisor_id,fsq_id,name,business_status,types,price_level,ta_price_level,rating,user_ratings_total,...,population_15_19,population_20_24,population_25_34,population_35_44,population_45_54,population_54_64,population_65+,resident_per_km2,brut_income_annual_per_person,cluster
0,ChIJd6kh67ZkjEcRC1IQourQOyo,7159041.0,4de685281f6e9d90c7f5325c,Restaurant La Tentation,OPERATIONAL,"['restaurant', 'point_of_interest', 'food', 'e...",2.0,2.0,4.2,158.0,...,1547,1937,5163,5549,4924,4290,5625,16282,109266,2
1,ChIJX68uubZkjEcRKKxpZ7OrgqU,13322506.0,NaN,Café Restaurant Palma,OPERATIONAL,"['restaurant', 'point_of_interest', 'food', 'e...",NaN,NaN,3.7,141.0,...,1547,1937,5163,5549,4924,4290,5625,16282,109266,2
2,ChIJPcYsG_BljEcR4kTFqBJJoK4,6720272.0,NaN,Le Versailles,OPERATIONAL,"['restaurant', 'point_of_interest', 'food', 'e...",NaN,2.0,3.9,56.0,...,1547,1937,5163,5549,4924,4290,5625,16282,109266,2
3,ChIJhb4e-Ep7jEcRY38kziIGZmE,NaN,NaN,Restaurant Divan,OPERATIONAL,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,NaN,4.5,113.0,...,1956,2148,4721,5127,5017,3956,5342,7187,123472,2
4,ChIJV8uySFxljEcRn7XpIO3zZX0,NaN,NaN,Restaurant Les Jardins du Rhône,OPERATIONAL,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,NaN,NaN,NaN,...,1547,1937,5163,5549,4924,4290,5625,16282,109266,2


In [99]:
df_rest.columns

Index(['place_id', 'tripadvisor_id', 'fsq_id', 'name', 'business_status',
       'types', 'price_level', 'ta_price_level', 'rating',
       'user_ratings_total', 'ta_rating', 'ta_user_ratings_total',
       'combined_rating', 'geometry.location.lat', 'geometry.location.lng',
       'formatted_address', 'zip_code', 'city', 'international_phone_number',
       'website', 'google_category', 'ta_4sq_cuisine1', 'ta_4sq_cuisine2',
       'ta_4sq_cuisine3', 'ta_4sq_cuisine4', 'ta_4sq_cuisine5',
       'main_category_1', 'main_category_2', 'main_category_3',
       'main_category_4', 'combined_main_category', 'sub_category',
       'neighboorhood_area_km2', 'resident_population_total',
       'population_age_0_14', 'population_15_19', 'population_20_24',
       'population_25_34', 'population_35_44', 'population_45_54',
       'population_54_64', 'population_65+', 'resident_per_km2',
       'brut_income_annual_per_person', 'cluster'],
      dtype='object')

In [100]:
tmp = df_rest[df_rest['price_level'] > 0]
len(tmp)

592

In [101]:
tmp = df_rest[(df_rest['price_level'] > 0) | (df_rest['ta_price_level'] > 0)]
len(tmp)

910

In [102]:
df_rest.isnull().sum() / len(df_rest) * 100

place_id                          0.000000
tripadvisor_id                   40.545567
fsq_id                           58.028518
name                              0.000000
business_status                   0.000000
types                             0.000000
price_level                      63.298202
ta_price_level                   50.960942
rating                           13.639182
user_ratings_total               13.639182
ta_rating                        44.451333
ta_user_ratings_total            44.451333
combined_rating                  12.213267
geometry.location.lat             0.000000
geometry.location.lng             0.000000
formatted_address                 0.000000
zip_code                          0.000000
city                              0.000000
international_phone_number       13.577185
website                          59.702418
google_category                   0.000000
ta_4sq_cuisine1                   0.371978
ta_4sq_cuisine2                   4.091754
ta_4sq_cuis

In [103]:
# Finding total categories
df_categories = pd.DataFrame()
df_categories['restaurant_category'] = df_rest[['main_category_1',
                                               'main_category_2',
                                               'main_category_3',
                                               'main_category_4']]\
                                                .apply(pd.Series.value_counts)\
                                                .sum(axis=1)\
                                                .sort_values(ascending=False)\
                                                .astype(int)
list_categories = df_categories.index.to_list()

In [117]:
list_categories

['General / Restaurant',
 'European',
 'French',
 'Italian',
 'General / Fast food / Snacks / Take Away',
 'Pizza',
 'Swiss',
 'General / Bar / Pub / Bistro',
 'Japanese',
 'General / Café',
 'Thai',
 'Chinese',
 'American',
 'Hamburger',
 'Lebanese',
 'Other Asian',
 'Turkish',
 'South American',
 'Indian',
 'Steakhouse / Barbecue / Grill',
 'Mexican',
 'Portuguese',
 'African',
 'Seafood',
 'Spanish',
 'Vegan / Vegetarian / Salad',
 'Other Middle Eastern',
 'Chicken',
 'Hawaiian',
 'All Other',
 '0']

In [105]:
df_regression = df_rest[(df_rest['price_level'] > 0) & (df_rest['combined_rating'] > 0) & (df_rest['combined_main_category'] != 0)]\
    [['geometry.location.lat','geometry.location.lng','price_level','combined_main_category','combined_rating']]
df_regression

,geometry.location.lat,geometry.location.lng,price_level,combined_main_category,combined_rating
0,46.205232,6.114945,2.0,"European, Swiss",4.1
3,46.196924,6.115081,2.0,General / Restaurant,4.5
6,46.210396,6.114510,2.0,General / Restaurant,3.9
7,46.210587,6.113854,2.0,General / Restaurant,4.3
9,46.220396,6.114030,2.0,"Swiss, European",4.1
...,...,...,...,...,...
1606,46.207746,6.169275,3.0,General / Restaurant,4.4
1608,46.192811,6.169873,2.0,General / Fast food / Snacks / Take Away,4.0
1609,46.199892,6.174117,2.0,American,4.1
1610,46.199773,6.174561,2.0,Swiss,4.3


In [120]:
list_categories[0:-1]

['General / Restaurant',
 'European',
 'French',
 'Italian',
 'General / Fast food / Snacks / Take Away',
 'Pizza',
 'Swiss',
 'General / Bar / Pub / Bistro',
 'Japanese',
 'General / Café',
 'Thai',
 'Chinese',
 'American',
 'Hamburger',
 'Lebanese',
 'Other Asian',
 'Turkish',
 'South American',
 'Indian',
 'Steakhouse / Barbecue / Grill',
 'Mexican',
 'Portuguese',
 'African',
 'Seafood',
 'Spanish',
 'Vegan / Vegetarian / Salad',
 'Other Middle Eastern',
 'Chicken',
 'Hawaiian',
 'All Other']

In [121]:
tmpx = pd.DataFrame()
tmpx['main_category'] = df_regression['combined_main_category']
for rest_type in list_categories[0:-1]:
    tmpx[rest_type] = 0

In [122]:
tmpx = tmpx.reset_index(drop=True)

In [123]:
tmpx

,main_category,General / Restaurant,European,French,Italian,General / Fast food / Snacks / Take Away,Pizza,Swiss,General / Bar / Pub / Bistro,Japanese,...,Mexican,Portuguese,African,Seafood,Spanish,Vegan / Vegetarian / Salad,Other Middle Eastern,Chicken,Hawaiian,All Other
0,"European, Swiss",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,General / Restaurant,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,General / Restaurant,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,General / Restaurant,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Swiss, European",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587,General / Restaurant,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
588,General / Fast food / Snacks / Take Away,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
589,American,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
590,Swiss,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [110]:
cat_row = tmpx.iloc[0]['main_category'].split(', ')
cat_row

['European', 'Swiss']

In [111]:
tmpx.loc[0, cat_row] = 1

In [124]:
tmpx.loc[0]

main_category                               European, Swiss
General / Restaurant                                      0
European                                                  0
French                                                    0
Italian                                                   0
General / Fast food / Snacks / Take Away                  0
Pizza                                                     0
Swiss                                                     0
General / Bar / Pub / Bistro                              0
Japanese                                                  0
General / Café                                            0
Thai                                                      0
Chinese                                                   0
American                                                  0
Hamburger                                                 0
Lebanese                                                  0
Other Asian                             

In [125]:
tmpx.loc[0, 'main_category']

'European, Swiss'

In [126]:
for idx in range(len(tmpx)):
    cat_row = tmpx.loc[idx, 'main_category'].split(", ")
    tmpx.loc[idx, cat_row] = 1

In [127]:
tmpx

,main_category,General / Restaurant,European,French,Italian,General / Fast food / Snacks / Take Away,Pizza,Swiss,General / Bar / Pub / Bistro,Japanese,...,Mexican,Portuguese,African,Seafood,Spanish,Vegan / Vegetarian / Salad,Other Middle Eastern,Chicken,Hawaiian,All Other
0,"European, Swiss",0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,General / Restaurant,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,General / Restaurant,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,General / Restaurant,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Swiss, European",0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587,General / Restaurant,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
588,General / Fast food / Snacks / Take Away,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
589,American,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
590,Swiss,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


# Creating Restaurant Category Clusters

In [130]:
# creating a dictionary for limiting restaurant categories

rest_keys = [
    'General / Restaurant',
    'European',
    'French',
    'Italian',
    'General / Fast food / Snacks / Take Away',
    'Pizza',
    'Swiss',
    'General / Bar / Pub / Bistro',
    'Japanese',
    'General / Café',
    'Thai',
    'Chinese',
    'American',
    'Hamburger',
    'Lebanese',
    'Other Asian',
    'Turkish',
    'South American',
    'Indian',
    'Steakhouse / Barbecue / Grill',
    'Mexican',
    'Portuguese',
    'African',
    'Seafood',
    'Spanish',
    'Vegan / Vegetarian / Salad',
    'Other Middle Eastern',
    'Chicken',
    'Hawaiian',
    'All Other']

rest_values = [
    'General',
    'European',
    'European',
    'European',
    'General',
    'Pizza',
    'European',
    'General',
    'Asian',
    'General',
    'Asian',
    'Asian',
    'American',
    'Hamburger',
    'Middle Eastern',
    'Asian',
    'Middle Eastern',
    'South American',
    'Indian',
    'Steakhouse / Barbecue / Grill',
    'Mexican',
    'European',
    'African',
    'Seafood',
    'European',
    'Vegan / Vegetarian / Salad',
    'Middle Eastern',
    'Chicken',
    'Hawaiian',
    'All Other']

rest_dict = {rest_keys[i]: rest_values[i] for i in range(len(rest_keys))}
rest_dict

{'General / Restaurant': 'General',
 'European': 'European',
 'French': 'European',
 'Italian': 'European',
 'General / Fast food / Snacks / Take Away': 'General',
 'Pizza': 'Pizza',
 'Swiss': 'European',
 'General / Bar / Pub / Bistro': 'General',
 'Japanese': 'Asian',
 'General / Café': 'General',
 'Thai': 'Asian',
 'Chinese': 'Asian',
 'American': 'American',
 'Hamburger': 'Hamburger',
 'Lebanese': 'Middle Eastern',
 'Other Asian': 'Asian',
 'Turkish': 'Middle Eastern',
 'South American': 'South American',
 'Indian': 'Indian',
 'Steakhouse / Barbecue / Grill': 'Steakhouse / Barbecue / Grill',
 'Mexican': 'Mexican',
 'Portuguese': 'European',
 'African': 'African',
 'Seafood': 'Seafood',
 'Spanish': 'European',
 'Vegan / Vegetarian / Salad': 'Vegan / Vegetarian / Salad',
 'Other Middle Eastern': 'Middle Eastern',
 'Chicken': 'Chicken',
 'Hawaiian': 'Hawaiian',
 'All Other': 'All Other'}

In [133]:
set(rest_dict.values())

17

In [152]:
tmpy = pd.DataFrame()
tmpy['main_category'] = df_regression['combined_main_category']
for rest_type in set(rest_dict.values()):
    tmpy[rest_type] = 0
tmpy.reset_index(drop=True, inplace=True)

In [153]:
for idx in range(len(tmpy)):
    cat_row = tmpy.loc[idx, 'main_category'].split(", ")
    for cat in cat_row:
        tmpy.loc[idx, rest_dict[cat]] = 1

In [154]:
tmpy

,main_category,Seafood,Vegan / Vegetarian / Salad,Middle Eastern,Steakhouse / Barbecue / Grill,Hawaiian,General,American,Indian,European,Hamburger,African,Asian,Pizza,South American,Chicken,All Other,Mexican
0,"European, Swiss",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,General / Restaurant,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,General / Restaurant,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,General / Restaurant,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,"Swiss, European",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587,General / Restaurant,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
588,General / Fast food / Snacks / Take Away,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
589,American,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
590,Swiss,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


# Running Regression Analysis

In [209]:
df_regression = df_rest[(df_rest['price_level'] > 0) & (df_rest['combined_rating'] > 0) & (df_rest['combined_main_category'] != 0)]\
    [['geometry.location.lat','geometry.location.lng','price_level','combined_main_category','combined_rating']].reset_index(drop=True)
df_regression.head()

,geometry.location.lat,geometry.location.lng,price_level,combined_main_category,combined_rating
0,46.205232,6.114945,2.0,"European, Swiss",4.1
1,46.196924,6.115081,2.0,General / Restaurant,4.5
2,46.210396,6.114510,2.0,General / Restaurant,3.9
3,46.210587,6.113854,2.0,General / Restaurant,4.3
4,46.220396,6.114030,2.0,"Swiss, European",4.1


In [210]:
for rest_type in set(rest_dict.values()):
    df_regression[rest_type] = 0

for idx in range(len(df_regression)):
    row_category = df_regression.loc[idx, 'combined_main_category'].split(", ")
    for category in row_category:
        df_regression.loc[idx, rest_dict[category]] = 1
        
df_regression.drop(columns=['combined_main_category'], inplace=True)

In [211]:
from sklearn.preprocessing import MinMaxScaler

# Instanciate MinMax Scaler 
mm_scaler = MinMaxScaler()

# select features and fit
features = [
    'geometry.location.lat',
    'geometry.location.lng']

mm_scaler.fit(df_regression[features])

# scale
df_regression[features] = mm_scaler.transform(df_regression[features])

In [206]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False) # Instantiate encoder

ohe.fit(df_regression[['price_level']]) # Fit encoder
encoded = ohe.transform(df_regression[['price_level']]) # Encode / Transform GarageFinish
print(encoded)
print(ohe.categories_)

[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 ...
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]
[array([1., 2., 3., 4.])]


In [212]:
df_regression['price_level_1'], df_regression['price_level_2'], df_regression['price_level_3'], df_regression['price_level_4'] = encoded.T
df_regression.head()

,geometry.location.lat,geometry.location.lng,price_level,combined_rating,Seafood,Vegan / Vegetarian / Salad,Middle Eastern,Steakhouse / Barbecue / Grill,Hawaiian,General,...,Asian,Pizza,South American,Chicken,All Other,Mexican,price_level_1,price_level_2,price_level_3,price_level_4
0,0.484261,0.019853,2.0,4.1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,1.0,0.0,0.0
1,0.307162,0.022085,2.0,4.5,0,0,0,0,0,1,...,0,0,0,0,0,0,0.0,1.0,0.0,0.0
2,0.594350,0.012711,2.0,3.9,0,0,0,0,0,1,...,0,0,0,0,0,0,0.0,1.0,0.0,0.0
3,0.598420,0.001937,2.0,4.3,0,0,0,0,0,1,...,0,0,0,0,0,0,0.0,1.0,0.0,0.0
4,0.807508,0.004836,2.0,4.1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,1.0,0.0,0.0


In [248]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate

# define X and y
X = df_regression.drop(columns=['combined_rating','price_level'])
y = df_regression['combined_rating']

# Instantiate model 
model = LinearRegression() 

# 10-Fold Cross validate model 
cv_results = cross_validate(model, X, y, cv=10, scoring=['neg_mean_squared_error'])
cv_results

{'fit_time': array([0.01574492, 0.03063893, 0.01366401, 0.00649381, 0.02079177,
        0.00656796, 0.01476002, 0.00404692, 0.14500904, 0.00571012]),
 'score_time': array([0.00604081, 0.00257397, 0.00445914, 0.00400829, 0.00305605,
        0.00332022, 0.00517988, 0.00230694, 0.00338197, 0.02515197]),
 'test_neg_mean_squared_error': array([-0.0979626 , -0.08275897, -0.08192212, -0.1177427 , -0.09015041,
        -0.07207331, -0.06624049, -0.06311433, -0.07563396, -0.07640964])}

In [252]:
np.sqrt(abs(cv_results['test_neg_mean_squared_error'].mean()))

0.2870554861866533

In [234]:
cv_results

{'fit_time': array([0.24365211, 0.35412312, 0.19588518, 0.21962094, 0.24735999,
        0.22176313, 0.21953392, 0.19442606, 0.17393398, 0.18688011]),
 'score_time': array([0.01135802, 0.01688695, 0.00876808, 0.00840998, 0.02368784,
        0.00980186, 0.01097608, 0.00786114, 0.0079031 , 0.00931191]),
 'test_score': array([-0.19964774, -0.30692559, -0.18929564, -0.28026793,  0.03802218,
         0.0745098 , -0.01474201, -0.30585264, -0.239     , -0.48731219])}

In [227]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import VotingRegressor, StackingRegressor, RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

In [219]:
model = KNeighborsRegressor()
cv_results = cross_validate(model, X, y, cv=10)
score =  cv_results['test_score'].mean()

-0.12752565259075604

In [221]:
model = RandomForestRegressor(max_depth=50,min_samples_leaf=20)
cv_results = cross_validate(model, X, y, cv=10)
score =  cv_results['test_score'].mean()
score

-0.04460144320494508

In [224]:
model = XGBRegressor()
cv_results = cross_validate(model, X, y, cv=10)
score = cv_results['test_score'].mean()
score

-0.3514250071563606

In [226]:
model = AdaBoostRegressor(
    base_estimator=DecisionTreeRegressor(max_depth=None))

cv_results = cross_validate(model, X, y, cv=10)
score = cv_results['test_score'].mean()
score

-0.1910511756720533

In [245]:
X

,geometry.location.lat,geometry.location.lng,Seafood,Vegan / Vegetarian / Salad,Middle Eastern,Steakhouse / Barbecue / Grill,Hawaiian,General,American,Indian,...,Asian,Pizza,South American,Chicken,All Other,Mexican,price_level_1,price_level_2,price_level_3,price_level_4
0,0.484261,0.019853,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,1.0,0.0,0.0
1,0.307162,0.022085,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0.0,1.0,0.0,0.0
2,0.594350,0.012711,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0.0,1.0,0.0,0.0
3,0.598420,0.001937,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0.0,1.0,0.0,0.0
4,0.807508,0.004836,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587,0.537853,0.912303,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0.0,0.0,1.0,0.0
588,0.219484,0.922133,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0.0,1.0,0.0,0.0
589,0.370427,0.991839,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0.0,1.0,0.0,0.0
590,0.367882,0.999133,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,1.0,0.0,0.0


In [253]:
X.columns

Index(['geometry.location.lat', 'geometry.location.lng', 'Seafood',
       'Vegan / Vegetarian / Salad', 'Middle Eastern',
       'Steakhouse / Barbecue / Grill', 'Hawaiian', 'General', 'American',
       'Indian', 'European', 'Hamburger', 'African', 'Asian', 'Pizza',
       'South American', 'Chicken', 'All Other', 'Mexican', 'price_level_1',
       'price_level_2', 'price_level_3', 'price_level_4'],
      dtype='object')

In [255]:
# define X and y
X = df_regression.drop(columns=['combined_rating','price_level'])
y = df_regression['combined_rating']

# Instantiate model 
model = LinearRegression() 

reg = model.fit(X, y)

In [258]:
reg.score(X,y)

0.05935111087421663